В этом задании вам необходимо:

Реализовать t-критерий Стьюдента для 2 независимых выборок при условии неизвестной дисперсии. Необходимо считать значение статистики и p_value для разных видов гипотез (двусторонняя, односторонние), а также результат (отвергается нулевая гипотеза или нет). Сравнить результаты с реализацией в scipy.stats.ttest_ind. 

Реализовать bootstrap для оценки (через доверительные интервалы) медианных и средних значений распределений. Рассчитать для распределений: нормальное, экспоненциальное, смесь нормальных (параметры этих распределений на ваш выбор)
Рассчитать мощность критерия для t-критерия и критерия Манна-Уитни для разных распределений и разном эффекте (распределение и эффект на выбор, 2 минимум). 

Оценить корректность t-критерия и критерия Манна-Уитни на 2-х разных распределениях.

Пример распределений: нормальные, смеси нормальных, экспоненциальные, биномиальные.

In [15]:
import numpy as np
import pandas as pd
from scipy import stats
from tqdm import tqdm  # Загрузка tqdm для отображения прогресса в циклах

# Функция для бутстрэп оценок медианы и среднего
def bootstrap(data, num_samples, statistic_func, alpha):
    """
    Вычисляет доверительные интервалы для заданной статистики с помощью бутстрепа.

    Args:
        data (array): Исходный массив данных.
        num_samples (int): Количество бутстреп-выборок.
        statistic_func (callable): Функция статистики (например, np.mean или np.median).
        alpha (float): Уровень значимости.

    Returns:
        tuple: Нижняя и верхняя границы доверительного интервала.
    """
    
    # Генерируем множество выборок и вычисляем статистику для каждой выборки
    bootstrapped_stats = [statistic_func(np.random.choice(data, size=len(data))) for _ in range(num_samples)]
    lower = np.percentile(bootstrapped_stats, 100 * alpha / 2)
    upper = np.percentile(bootstrapped_stats, 100 * (1 - alpha / 2))
    
    return lower, upper

# Функция для выполнения t-теста
def ttest_2samp(x1, x2, alternative='two-sided'):
    """
    Выполняет двухвыборочный t-тест Стьюдента для независимых выборок.

    Args:
        x1 (array): Первая выборка наблюдений.
        x2 (array): Вторая выборка наблюдений.
        alternative (str): Определяет альтернативную гипотезу ('two-sided', 'less', 'greater').

    Returns:
        float: t-статистика.
        float: p-значение.
        bool: Результат теста (True если нулевая гипотеза отвергается).
    """
    
    n1, n2 = len(x1), len(x2)
    s1, s2 = x1.var(ddof=1), x2.var(ddof=1)
    x_mean1, x_mean2 = x1.mean(), x2.mean()
    
    # Степени свободы
    df = n1 + n2 - 2
    
    # Объединенная оценка дисперсии
    sp2 = ((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / df
    
    # Стандартная ошибка и t-статистика
    SE = np.sqrt(s1/n1 + s2/n2)
    t_statistic = (x_mean1 - x_mean2) / SE
    dof = (s1/n1 + s2/n2)**2 / (s1**2/(n1**2*(n1-1)) + s2**2/(n2**2*(n2-1)))
   
    
    # Выбор альтернативной гипотезы и расчет p-значения
    if alternative == 'two-sided':
        p_value = stats.t.sf(np.abs(t_statistic), dof) * 2
    elif alternative == 'less':
        p_value = stats.t.cdf(t_statistic, dof)
    elif alternative == 'greater':
        p_value = stats.t.sf(t_statistic, dof)
    
    # Отвержение нулевой гипотезы на уровне значимости 0.05
    reject_null = p_value < 0.05
    
    return t_statistic, p_value, reject_null

# Пример использования функции bootstrap для оценки среднего и медианы для нормального распределения
data_normal = np.random.normal(0, 1, 1000)

# Среднее значение
mean_confidence_interval = bootstrap(data_normal, 10000, np.mean, 0.05)
print(f'Bootstrap confidence interval for the mean (normal distribution):   {mean_confidence_interval[0]:.4f}, {mean_confidence_interval[1]:.4f}')

# Медиана
median_confidence_interval = bootstrap(data_normal, 10000, np.median, 0.05)
print(f'Bootstrap confidence interval for the median (normal distribution): {median_confidence_interval[0]:.4f}, {median_confidence_interval[1]:.4f}')


Bootstrap confidence interval for the mean (normal distribution):   -0.0829, 0.0373
Bootstrap confidence interval for the median (normal distribution): -0.1474, 0.0064


In [2]:
from scipy.stats import ttest_ind, mannwhitneyu

def simulate_ttest(dist1_params, dist2_params, num_simulations, alpha, effect_size):
    power_count = 0
    type1_error_count = 0
    for _ in range(num_simulations):
        # Генерация выборок
        sample1 = np.random.normal(**dist1_params)
        sample2 = np.random.normal(**dist2_params)
        
        # Расчёт эффекта (если нужен)
        sample2 += effect_size

        # Проведение t-теста
        stat, p_value = ttest_ind(sample1, sample2)

        # Проверка на значимость
        if p_value < alpha:
            power_count += 1
        
        # Проверка значимости без эффекта (для ошибок 1-го рода)
        stat, p_value = ttest_ind(sample1, np.random.normal(**dist2_params))
        if p_value < alpha:
            type1_error_count += 1
    
    # Расчет мощности и уровня ошибок первого рода
    power = power_count / num_simulations
    type1_error_rate = type1_error_count / num_simulations

    return power, type1_error_rate

# Параметры для симуляции
dist1_params = {'loc': 0, 'scale': 1, 'size': 100}  # Параметры для группы A
dist2_params = {'loc': 0, 'scale': 1, 'size': 100}  # Параметры для группы B
num_simulations = 10000
alpha = 0.05
effect_size = 0.5  # Размер эффекта

# Запуск симуляции для t-теста
power, type1_error_rate = simulate_ttest(dist1_params, dist2_params, num_simulations, alpha, effect_size)
print("Estimated power:", power)
print("Type I error rate:", type1_error_rate)

Estimated power: 0.9399
Type I error rate: 0.0498


In [3]:
from scipy.stats import mannwhitneyu

def simulate_mannwhitneyu(num_simulations, alpha, effect_size):
    power_count = 0
    type1_error_count = 0
    for _ in range(num_simulations):
        # Генерируем выборки без эффекта
        sample1_no_effect = np.random.normal(loc=0, scale=1, size=30)
        sample2_no_effect = np.random.normal(loc=0, scale=1, size=30)
        
        # Проверяем ошибку первого рода
        stat_no_effect, p_value_no_effect = mannwhitneyu(sample1_no_effect, sample2_no_effect, alternative='two-sided')
        if p_value_no_effect < alpha:
            type1_error_count += 1
            
        # Генерируем выборки с эффектом
        sample1_with_effect = np.random.normal(loc=0, scale=1, size=30)
        sample2_with_effect = np.random.normal(loc=effect_size, scale=1, size=30)  # добавляем эффект к среднему
        
        # Проверяем мощность теста (нулевая гипотеза неверна)
        stat_with_effect, p_value_with_effect = mannwhitneyu(sample1_with_effect, sample2_with_effect, alternative='two-sided')
        if p_value_with_effect < alpha:
            power_count += 1
    
    power_estimate = power_count / num_simulations
    type1_error_estimate = type1_error_count / num_simulations
    
    return power_estimate, type1_error_estimate

num_simulations = 10000  # Количество симуляций для точной оценки
alpha = 0.05  # Уровень значимости
effect_size = 0.5  # Примерный размер эффекта, который мы хотим обнаружить

power_estimate_mw, type1_error_estimate_mw = simulate_mannwhitneyu(
    num_simulations,
    alpha,
    effect_size
)

print(f"Estimated power (Mann-Whitney U): {power_estimate_mw}")
print(f"Type I error rate (Mann-Whitney U): {type1_error_estimate_mw}")

Estimated power (Mann-Whitney U): 0.455
Type I error rate (Mann-Whitney U): 0.0529


In [4]:
from statsmodels.stats.power import tt_ind_solve_power

def calculate_power(n1, mean1, std1, n2, mean2, std2, alpha=0.05, alternative='two-sided'):
    effect_size = (mean1 - mean2) / np.sqrt((std1**2 + std2**2) / 2)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=n1,
                               ratio=n2/n1,
                               alpha=alpha,
                               alternative=alternative)
    return power

In [5]:
from scipy.stats import mannwhitneyu

def mann_whitney_test(x1, x2, alternative='two-sided'):
    stat, p_value = mannwhitneyu(x1, x2, alternative=alternative)
    reject_null = p_value < 0.05
    return stat, p_value, reject_null

# Пример расчета мощности для нормально распределённых выборок
sample1 = np.random.normal(loc=0, scale=1, size=100)
sample2 = np.random.normal(loc=0.5, scale=1, size=100)

power_ttest = calculate_power(len(sample1), sample1.mean(), sample1.std(),
                              len(sample2), sample2.mean(), sample2.std())

stat, p_value, reject_null = mann_whitney_test(sample1, sample2)

print(f"Power of t-test: {power_ttest:.4f}")
print(f"Stat: {stat:.0f}, p-value: {p_value:.6f}, Reject Null: {reject_null}")

Power of t-test: 0.9673
Stat: 3477, p-value: 0.000199, Reject Null: True


In [6]:
# Генерация выборок из экспоненциального распределения
exp_sample1 = np.random.exponential(scale=1.0, size=100)
exp_sample2 = np.random.exponential(scale=1.5, size=100)

# Генерация выборок из смеси нормальных распределений
mix_normal_sample1 = np.concatenate([np.random.normal(0, 1, 50), np.random.normal(5, 1, 50)])
mix_normal_sample2 = np.concatenate([np.random.normal(0.5, 1, 50), np.random.normal(4.5, 1, 50)])

# Проверка t-критерия Стьюдента
t_stat_exp, p_value_exp, reject_null_exp = ttest_2samp(exp_sample1, exp_sample2)
t_stat_mix, p_value_mix, reject_null_mix = ttest_2samp(mix_normal_sample1, mix_normal_sample2)

print(f"Exponential: t_stat={t_stat_exp:.4f}, p_value={p_value_exp:.4f}, reject_null={reject_null_exp}")
print(f"Mixed Normal: t_stat={t_stat_mix:.4f}, p_value={p_value_mix:.4f}, reject_null={reject_null_mix}")

# Проверка t-критерия Манна-Уитни
mw_stat_exp, mw_p_value_exp, mw_reject_null_exp = mann_whitney_test(exp_sample1, exp_sample2)
mw_stat_mix, mw_p_value_mix, mw_reject_null_mix = mann_whitney_test(mix_normal_sample1, mix_normal_sample2)

print(f"Exponential Mann-Whitney: stat={mw_stat_exp:.0f}, p_value={mw_p_value_exp:.4f}, reject_null={mw_reject_null_exp}")
print(f"Mixed Normal Mann-Whitney: stat={mw_stat_mix:.0f}, p_value={mw_p_value_mix:.4f}, reject_null={mw_reject_null_mix}")

Exponential: t_stat=-3.0094, p_value=0.0030, reject_null=True
Mixed Normal: t_stat=0.2973, p_value=0.7665, reject_null=False
Exponential Mann-Whitney: stat=3562, p_value=0.0004, reject_null=True
Mixed Normal Mann-Whitney: stat=5183, p_value=0.6557, reject_null=False


In [7]:
from __future__ import print_function

import numpy as np
from scipy.special import stdtr
from scipy.stats import ttest_ind

a = np.random.randn(40)
b = 4*np.random.randn(40)
t, p = ttest_ind(a, b, equal_var=False)
print("ttest_ind:            t = %g  p = %g" % (t, p))

t1, p1, tf = ttest_2samp(a,b)
print("ttest_2samp:          t = %g  p = %g" % (t1, p1))

# Вычисляем описательную статистику a и b
x_mean1 = np.mean(a)
s1 = np.var(a, ddof=1)
n1 = a.size
adof = n1 - 1

x_mean2 = np.mean(b)
s2 = np.var(b, ddof=1)
n2 = b.size
bdof = n2 - 1
# Используем формулу напрямую
tf = (x_mean1 - x_mean2) / np.sqrt(s1/n1 + s2/n2)
dof = (s1/n1 + s2/n2)**2 / (s1**2/(n1**2*adof) + s2**2/(n2**2*bdof))
pf = 2*stdtr(dof, -np.abs(tf))

print("formula:              t = %g  p = %g" % (tf, pf))

ttest_ind:            t = 0.0867071  p = 0.931318
ttest_2samp:          t = 0.0867071  p = 0.931318
formula:              t = 0.0867071  p = 0.931318
